# CinematicAI: Intelligent Movie Recommendation Assistant

This is a sample notebook to demonstrate the structure of the full implementation. The complete notebook is too large to be included in this repository.

## Environment Setup

In [ ]:
!pip install sentence-transformers pinecone transformers pandas torch rank_bm25 nltk --quiet
!pip install nest_asyncio --quiet
!pip install -q -U bitsandbytes  # Required for 4-bit quantization
!pip install -U transformers

## Configuration & Imports

In [ ]:
import pandas as pd
import numpy as np
import torch
import re
import ast
import nltk
import asyncio
import nest_asyncio
import gc
import time
import joblib
import os
from datetime import datetime
from pinecone import Pinecone, ServerlessSpec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi

In [ ]:
# Free up memory before starting
torch.cuda.empty_cache()
gc.collect()

CONFIG = {
    # Core components
    "index_name": "movie-rag-index",
    "embedding_model": "sentence-transformers/all-mpnet-base-v2",
    "reranker": "cross-encoder/ms-marco-MiniLM-L-6-v2",
    "llm_model": "microsoft/Phi-3-mini-4k-instruct",

    # Processing parameters
    "chunk_size": 600,
    "chunk_overlap": 100,
    "top_k_retrieve": 50,
    "top_k_final": 12,

    # Response generation
    "max_response_tokens": 800,
    "min_imdb_rating": 6.5,
    "default_year_range": 10,
}

# Hardware configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

## Data Loading and Processing

In [ ]:
# Load the dataset
df = pd.read_csv('reviews.csv')
print(f"Dataset loaded with {len(df)} reviews")
df.head()

## Sample Content

The full notebook contains the following sections:

1. Data Preprocessing
2. BM25 Index Creation
3. Document Chunking
4. Vector Embedding
5. Pinecone Index Creation and Upload
6. Hybrid Retrieval Implementation
7. Re-ranking Implementation
8. Context Building
9. Response Generation
10. Query Interface

## Example RAG Assistant Function

The final RAG assistant would include code similar to the following:

In [ ]:
def get_movie_recommendations(user_query, chat_history=None):
    """
    Main function to generate movie recommendations based on user query
    
    Args:
        user_query (str): User's natural language request for movie recommendations
        chat_history (list, optional): Previous conversation turns for context
        
    Returns:
        str: Natural language response with movie recommendations
    """
    # 1. Process query
    query_intent, constraints = analyze_query(user_query, chat_history)
    
    # 2. Retrieve relevant information
    bm25_results = bm25_search(user_query, top_k=CONFIG["top_k_retrieve"])
    vector_results = vector_search(user_query, top_k=CONFIG["top_k_retrieve"])
    combined_results = combine_search_results(bm25_results, vector_results)
    
    # 3. Re-rank results
    reranked_results = rerank_results(user_query, combined_results)
    top_results = reranked_results[:CONFIG["top_k_final"]]
    
    # 4. Build context for LLM
    context = build_context(user_query, top_results, constraints)
    
    # 5. Generate response
    prompt = create_prompt(user_query, context, chat_history)
    response = generate_response(prompt)
    
    return response